In [3]:
import pandas as pd
import numpy as np

import warnings
import io,base64,os,json,re

import mariadb
import sys

import requests

import time
from datetime import date,datetime
from bs4 import BeautifulSoup as bs
from xml.dom.minidom import parse, parseString

warnings.filterwarnings('ignore')

In [4]:
import cx_Oracle as cx

import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import webbrowser

# initalise the tkinter GUI
root = tk.Tk()

root.geometry("400x300") # set the root dimensions
root.pack_propagate(False) # tells the root to not let the widgets inside it determine its size.
root.resizable(0, 0) # makes the root window fixed in size.

df=pd.DataFrame()



# Frame for open file dialog
file_frame = tk.LabelFrame(root, text="Arcep-points")
file_frame.place(height=200, width=400, relx=0)

# Buttons
upload_file=tk.Entry(file_frame,width=36)
upload_file.place(rely=0.20, relx=0.35)




upload_file.place(rely=0.20, relx=0.35)


Button1=tk.Button(file_frame, text="Browse A File", command=lambda: File_dialog())
Button1.place(rely=0.18, relx=0.10)




s = tk.Label(file_frame, text="nbre_neigh")
s.place(rely=0.35, relx=0.13, width=60)

neigh=tk.IntVar()
nbre_neigh = tk.Entry(root, textvariable=neigh)
nbre_neigh.place(rely=0.26, relx=0.35,width=50)

na = tk.Label(file_frame, text="Netact")
na.place(rely=0.48, relx=0.15, width=50)

liste_nect=["NA1", "NA2","NA3","NA6"]
listeCombo = ttk.Combobox(root, values=liste_nect)
listeCombo.current(1)


listeCombo.place(rely=0.35, relx=0.35,width=50)


f = tk.Label(file_frame, text="key_generator")
f.place(rely=0.66,relx=0.13)

key=tk.StringVar()
key_generator = tk.Entry(root, textvariable=key)
key_generator.place(rely=0.46, relx=0.35,width=200)


df_cluster=pd.DataFrame() 


button2 = tk.Button(file_frame, text="Load_Result", command=lambda: Load_data())
button2.place(rely=0.8, relx=0.4)

def callback(url):
   webbrowser.open_new_tab(url)

#Create a Label to display the link
link = tk.Label(file_frame, text="https://www.blitter.se/utils/basic-authentication-header-generator/",font=('Helveticabold', 9), fg="blue", cursor="hand2")
link.place(rely=0.8, relx=0.9)
link.pack()
link.bind("<Button-1>", lambda e:
callback("https://www.blitter.se/utils/basic-authentication-header-generator/"))


def File_dialog():
    
    global filename
    """This Function will open the file explorer and assign the chosen file path to label_file"""

    filename = filedialog.askopenfilename(initialdir="/",
                                          title="Select A File",
                                          filetype=(("csv files", "*.csv"),("All Files", "*.*")))
    upload_file.insert(tk.END, filename)
    return None
def Load_data():
    global df_cluster
    global netact
    df_cluster=pd.read_csv(filename,encoding="ISO-8859-1",delimiter=';')    
    liste=np.reshape([filename],(1,1))
    netact=listeCombo.get()
    
    root.destroy() 
    return None

root.mainloop()

In [128]:
df_cluster.head()
netact

'NA3'

In [129]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="fethi",
        password="cassis_2021",
        host="192.168.103.9",
        port=3306,
        database="op_gate"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()



In [130]:
#SQL queries
#retreive to LNCEL KPI
df=pd.read_sql("select * from lncel_kpis_na"+netact[-1]+" WHERE PERIOD_START_TIME >= now()-interval 8 day",conn)


In [131]:
#retreive flux HO
df_ho=pd.read_sql("select * from neighbor_ho_analysis  WHERE ((PERIOD_START_TIME IN (SELECT max(PERIOD_START_TIME) FROM neighbor_ho_analysis)) and (`Source LNCEL name` in"+str(tuple(df_cluster["name"]))+"))",conn)

In [132]:
#retreive Wind Cell
df_wind_sql=pd.read_sql("select * from wind_cell_state  WHERE DATE IN (SELECT max(DATE) FROM Wind_cell_state)",conn)


In [133]:
df_wind_sql.head()

,DATE,YEAR,WEEK,LNCEL name,avg_act_dl_ue,dl_avg_prb_util,DL_thpt_users,bh_hour,band,threshold_active_ue,...,dl_spect_efficiency,mapped_eff,pdsch_subf_nr_ratio,thput_dl_qci6_ca,congested_wind_rule,wind_count,neighb_avail_percent,sector,netact,availability
0,2023-12-11,2023,50,01004_001-51,0.6304,31.58,26.032,19.0,5,10.410000,...,2.082,1.0,0.0,0.0,0,NaN,None,01004_001_1,3,99.99
1,2023-12-11,2023,50,01004_001-52,9.0818,91.50,6.364,18.4,5,15.820000,...,3.164,1.0,0.0,0.0,0,NaN,None,01004_001_2,3,100.00
2,2023-12-11,2023,50,01004_001-61,1.1292,33.20,37.850,19.0,6,18.160000,...,2.724,1.0,0.0,0.0,0,NaN,None,01004_001_1,3,100.00
3,2023-12-11,2023,50,01004_001-62,0.8824,46.88,27.388,15.2,6,6.666667,...,0.922,1.0,0.0,0.0,0,NaN,None,01004_001_2,3,99.99
4,2023-12-11,2023,50,01004_001-71,2.0376,51.24,9.890,18.4,7,6.636653,...,2.954,1.0,32.6,0.0,0,NaN,None,01004_001_1,3,100.00


In [134]:
#calculate KPIS SR
for i in df.columns[df.columns.str.contains("sr")].str[:-2]:
    df[i]=df[i+'_n']/df[i+'_d']

In [135]:
#Drop duplicate num deno
for i in df.columns[df.columns.str.contains("sr_")]:  
    df.drop([i],axis=1,inplace=True)

In [136]:
#calculate KPIS SR (optionel)
df["e_rab_dr_ran_view"]=df["e_rab_dr_ran_view_n"]/df["e_rab_dr_ran_view_d"]

df["e_utran_cell_avail_ratio"]=df["e_utran_cell_avail_ratio_n"]/df["e_utran_cell_avail_ratio_d"]

df["avg_ip_sched_thp_ul_qci6"]=df["avg_ip_sched_thp_ul_qci6_n"]/df["avg_ip_sched_thp_ul_qci6_d"]
df["avg_ip_sched_thp_dl_qci6"]=df["avg_ip_sched_thp_dl_qci6_n"]/df["avg_ip_sched_thp_dl_qci6_d"]

df.drop(['e_rab_dr_ran_view_n','e_rab_dr_ran_view_d','avg_ip_sched_thp_dl_qci6_n','avg_ip_sched_thp_dl_qci6_d',\
        'avg_ip_sched_thp_ul_qci6_n','avg_ip_sched_thp_ul_qci6_d','e_utran_cell_avail_ratio_n','e_utran_cell_avail_ratio_d'],axis=1,inplace=True)


In [137]:
df_kpi_clus=df[df["LNCEL name"].isin(df_cluster["name"].unique())]

df_wind=df_wind_sql[df_wind_sql['congested_wind_rule']==1]


In [138]:
df_cluster.head()

,id,distName,name,eutraCelId,earfcnDL,earfcnUL,angle,tac,qrxlevmin,eCallAcBarred,...,prachFreqOff,prachHsFlag,a3Offset,threshold4,threshSrvLow,sIntrasearch,sNonIntrsearch,qHyst,qrxlevminintraF,netActId
0,99707,PLMN-PLMN/MRBTS-403059/LNBTS-403059/LNCEL-61,69190_001-61,103183165,3350,21350,80.0,6915,-126,1,...,8,0,6,16,4,62,20,4,-126,3
1,99708,PLMN-PLMN/MRBTS-403059/LNBTS-403059/LNCEL-62,69190_001-62,103183166,3350,21350,50.0,6915,-126,1,...,8,0,6,16,4,62,20,4,-126,3
2,99709,PLMN-PLMN/MRBTS-403059/LNBTS-403059/LNCEL-63,69190_001-63,103183167,3350,21350,60.0,6915,-126,1,...,8,0,6,16,4,62,20,4,-126,3
3,100802,PLMN-PLMN/MRBTS-407247/LNBTS-407247/LNCEL-61,69044_001-61,104255293,3350,21350,40.0,6913,-126,1,...,8,0,6,16,4,62,20,4,-126,3
4,100803,PLMN-PLMN/MRBTS-407247/LNBTS-407247/LNCEL-62,69044_001-62,104255294,3350,21350,40.0,6913,-126,1,...,8,0,6,16,4,62,20,4,-126,3


In [139]:
df_kpi_clus.loc[df_kpi_clus["LNCEL name"].isin(df_wind["LNCEL name"].unique()),"Wind-rule"]='1'

In [140]:
df_kpi_clus["Wind-rule"].unique()

array([nan, '1'], dtype=object)

In [141]:
#filter wind cell-flux HO
df_kpi_cio=df_kpi_clus[df_kpi_clus["Wind-rule"]=='1']

df_ho_fil=df_ho[df_ho["Source LNCEL name"].isin(df_kpi_cio["LNCEL name"].unique())]

df_ho_fil.sort_values(by="Inter eNB neighbor HO: Att",ascending=False,inplace=True)


In [142]:
df_wind_ho=pd.merge(df_ho_fil,df_wind_sql[["LNCEL name","dl_avg_prb_util","congested_wind_rule"]],left_on="Target LNCEL name",right_on="LNCEL name",how='left')

In [143]:
df_kpi_cio.head()

,PERIOD_START_TIME,MRBTS name,LNBTS name,LNCEL name,avg_prb_usage_tti_dl_n,avg_prb_usage_tti_ul_n,pdcp_sdu_vol_dl,pdcp_sdu_vol_ul,pdcp_sdu_vol_dl_x2,pdcp_sdu_vol_ul_x2,...,e_utran_interrat_ho_sr,erab_qci1_setup_sr,qci1_intrafreq_ho_sr,qci1_interfreq_ho_sr,srvcc_e_utran_interrat_ho_sr,e_rab_dr_ran_view,e_utran_cell_avail_ratio,avg_ip_sched_thp_ul_qci6,avg_ip_sched_thp_dl_qci6,Wind-rule
41082,2023-12-12,LTE-403414,LTE-69069-003,69069_003-71,459.0,288.0,63229.73,6587.61,564.37,1038.80,...,92.156863,99.802176,98.214286,88.135593,92.156863,0.105837,100.000000,303.346154,3461.881029,1
42229,2023-12-12,LTE-402461,LTE-69202-004,69202_004-83,538.0,122.0,94947.96,7188.34,10388.74,1092.53,...,88.281250,100.000000,100.000000,98.178506,88.281250,0.109733,100.000000,308.663529,4364.187057,1
45636,2023-12-12,3G-310366,LTE-69389-015,69389_015-52,874.0,262.0,127200.15,13968.14,25238.83,3149.39,...,92.592593,100.000000,99.769762,99.708879,92.592593,0.058965,73.293981,538.226902,1907.236721,1
45644,2023-12-12,LTE-410366,LTE-69389-015,69389_015-81,757.0,123.0,129397.69,9418.02,13400.81,1330.11,...,90.789474,100.000000,99.467377,97.474747,90.789474,0.198697,100.000000,336.884744,6120.080115,1
99240,2023-12-13,LTE-403414,LTE-69069-003,69069_003-71,507.0,317.0,68177.34,7876.64,677.48,1233.17,...,97.297297,100.000000,98.711755,69.911504,97.297297,0.098068,100.000000,328.337945,3478.846058,1


In [144]:
# exculde wind target
df_wind_ho.shape
df_wind_ho=df_wind_ho[df_wind_ho['congested_wind_rule']!=1]

In [145]:
df_wind_ho.head()

,PERIOD_START_TIME,Source LNCEL name,s_freq,Target LNCEL name,t_freq,HO_type,Intra eNB neighbor HO: Prep SR,Intra eNB neighbor HO: SR,Intra eNB neighbor HO: Att,Inter eNB neighbor HO: Prep SR,Inter eNB neighbor HO: SR,Inter eNB neighbor HO: Att,tot_rel_att,success_rate,total_att,share,eci_id,LNCEL name,dl_avg_prb_util,congested_wind_rule
0,2023-12-11,69389_015-81,8,69389_028-83,8,intra_freq,NaN,NaN,0,99.99,99.53,63281,63281,99.53,186378,33.953042,4327251,69389_028-83,79.24,0.0
1,2023-12-11,69389_015-52,5,69389_015-72,7,inter_freq,NaN,NaN,0,99.89,98.76,52339,52339,98.76,114918,45.544649,105053768,69389_015-72,17.08,0.0
2,2023-12-11,69389_015-52,5,69384_001-53,5,intra_freq,NaN,NaN,0,99.96,98.44,44343,44343,98.44,131635,33.686330,79611701,69384_001-53,94.32,0.0
3,2023-12-11,69202_004-83,8,69244_006-82,8,intra_freq,NaN,NaN,0,99.99,99.85,24238,24238,99.85,126930,19.095564,107681106,69244_006-82,59.54,0.0
4,2023-12-11,69389_015-81,8,69389_012-81,8,intra_freq,NaN,NaN,0,99.99,99.50,21863,21863,99.50,186378,11.730462,103602769,69389_012-81,37.36,0.0


In [146]:
df_grp=df_wind_ho.groupby('Source LNCEL name',group_keys=False).apply(lambda x: x.nlargest(neigh.get(),columns='Inter eNB neighbor HO: Att')).reset_index()

In [147]:
liste_cells=df_grp.groupby('Source LNCEL name')['Target LNCEL name'].apply(lambda x: x.values.tolist()).to_dict()

In [148]:
liste_cells



{'69069_003-71': ['69069_002-73', '69205_003-73', '69255_003-71'],
 '69202_004-83': ['69244_006-82', '69069_002-81', '69244_002-83'],
 '69389_015-52': ['69389_015-72', '69384_001-53', '69387_004-52'],
 '69389_015-81': ['69389_028-83', '69389_012-81', '69384_004-83']}

In [149]:
netact_in=netact
name_xml='dpt_sud'


In [150]:
# cellule a garder 11/04/2022

dict_blackliste={ 'handoverAllowed':'1','removeAllowed':'0'}
key_netact=key.get()
#key_netact="YW1la2VudGk6U2Ftc2ljMDEh"
#https://www.blitter.se/utils/basic-authentication-header-generator/
dict_url={'NA1':"http://na1waslb.nct8.mob.proxad.net/netact/cm/open-api/persistency/v1/query",
          'NA2':"http://na2waslb.nct15.mob.proxad.net/netact/cm/open-api/persistency/v1/query",
          'NA3':"http://na3waslb.nct16.mob.proxad.net/netact/cm/open-api/persistency/v1/query",
          "NA6":"http://na6waslb.nct6.mob.proxad.net/netact/cm/open-api/persistency/v1/query"}

In [151]:
#get the xml format of the output
def to_xml_liste_df(liste_df,name_xml,netact):
    
    def to_tag_managedObject(row):
        tag_managedObject=soup.new_tag("managedObject",
                               class_=row.mo ,version=row.version, distName=row.dn, operation=row.action)
        return tag_managedObject
    
    def to_tag_p(param,value_param):
        tag_p=soup.new_tag('p')
        tag_p['name']=f'{param}'
        tag_p.string=f'{value_param}'
        return tag_p
    
    soup =bs(features='xml')
    tag_raml =soup.new_tag('raml', xmlns="raml20.xsd", version="2.0")
    tag_cmData =soup.new_tag('cmData', type="plan" ,scope="all")
    tag_cmData['name']="Free_Mobile"
    tag_header =soup.new_tag('header')
    dt = str(datetime.now())
    datetime_ = dt.replace(' ', 'T').split('.')[0]
    tag_log=soup.new_tag('log',dateTime=f"{datetime_}", action="created",appInfo="ActualExporter")
    tag_log.string='InternalValues are used'
    tag_header.insert(1,tag_log)
    tag_cmData.insert(1,tag_header)
    i=2
    for df in liste_df:
        if len(df):
            for row in df.itertuples(): 
                j=1
                tag_managedObject=to_tag_managedObject(row)
                for param in row.param_inco.split(','):
                    tag_managedObject.insert(j,to_tag_p(param,getattr(row, param)))
                    j=j+1
                i=i+1
                tag_cmData.insert(i,tag_managedObject)
    tag_raml.insert(1,tag_cmData)
    soup.insert(1,tag_raml)
    final_xml = parseString(soup.encode('utf-8')).toprettyxml()
    with open(f'Plan_Optim_{name_xml}_{netact}_{date.today()}.xml', "w") as text_file:
            text_file.write(final_xml)
    with open(f'Plan_Optim_{name_xml}_{netact}_{date.today()}.xml') as xmlfile:
        lines = [line for line in xmlfile if line.strip() != '']
        lines=[line.replace('class_','class') for line in lines]
        lines.insert(1, "<!DOCTYPE raml SYSTEM 'raml20.dtd'>\n")
    with open(f'Plan_Optim_{name_xml}_{netact}_{date.today()}.xml', "w") as xmlfile:
        xmlfile.writelines(lines)
    return 

In [152]:
# getting the lnrel of the selected eNodeb
def get_lnrel(target_eNB):
    url=dict_url[netact_in]
    mopath_obj = f"\"PLMN/MRBTS/NOKLTE:LNBTS/NOKLTE:LNCEL/NOKLTE:LNREL [@ecgiAdjEnbId={target_eNB}]\""
    expression="[\"dn()\",\"version()\",\"@ecgiLcrId\",\"@ecgiAdjEnbId\",\"@cellIndOffNeigh\",\"@handoverAllowed\",\"@removeAllowed\"]"
    requete_param = "{\"confId\": 1,\"moPath\": "+mopath_obj+",\"expressions\": "+expression+"}"
    headers = {
                'Authorization': f'Basic {key_netact}',
                'Content-Type': 'application/json'
            }
    response = requests.request("POST", url, headers=headers, data=requete_param)
    #response = requests.request("POST", url, headers=headers, data=payload)
    cols=["dn","version","ecgiLcrId","ecgiAdjEnbId","cellIndOffNeigh","handoverAllowed","removeAllowed"]
    if response.status_code == 200:
        df=pd.DataFrame(response.json()['result'], dtype='str', columns=cols)
        df['dn_parent']=df.dn.str.split(f'/LNREL-').str[0]
        df['param_inco']=''
        df['action']='update'
        df['check']='NOK'
        #df['dn_id']=df.dn.str.split(f'/').str[-1]
        df['ecgiSourceEnbId']=df.dn.str.split(f'/LNCEL-').str[0]
        df['ecgiSourceEnbId']=df.ecgiSourceEnbId.str.split(f'/LNBTS-').str[1]
    return df

In [153]:
#get LNCEL Info 
liste_LNCEL=[]
for netact in['NA1','NA2','NA3','NA6']:
    url=dict_url[netact]
    dict_obj_mopath={'NOKLTE:LNCEL': 'PLMN/MRBTS/NOKLTE:LNBTS/'}
   
    #on definit la liste des param en cours
    list_para_obj=['dn()','version()','name','eutraCelId','lcrId','administrativeState']
    obj='NOKLTE:LNCEL'

    # on formate l expression a base de parametres
    expressions_para=[f'\"{x}\"' if i<2 else f'\"@{x}\"' for i,x in enumerate(list_para_obj)]

    mopath_obj = f"\"{dict_obj_mopath[obj]}{obj } \""
    expression=str(expressions_para).replace('\'','')
    requete_param = "{\"confId\": 1,\"moPath\": "+mopath_obj+",\"expressions\": "+expression+"}"

    headers = {
                'Authorization': f'Basic {key_netact}',
                'Content-Type': 'application/json'
            }
    response = requests.request("POST", url, headers=headers, data=requete_param)
    print(response.status_code)
    if response.status_code == 200:
        df=pd.DataFrame(response.json()['result'], dtype='str', columns=[x.replace('()','') for x in list_para_obj])
        mo=obj.split(':')[-1]
        df['mo']=mo
        df['dn_parent']=df.dn.str.split(f'/{mo}').str[0]
        df['param_inco']=''
        df['action']='update'
        df['check']='OK'
        df['netact']=netact
        for col in df.columns :
            df[col] = df[col].astype(str).str.split('.').str[0]
        exec('{} = df'.format(obj.split(':')[-1]))
            
        liste_LNCEL.append(df)

200
200
200
200


In [154]:
LNCEL=pd.concat(liste_LNCEL,sort=False)

In [155]:
LNCEL['ecgiSourceEnbId']=LNCEL.dn_parent.str.split(f'/LNBTS-').str[1]
LNCEL['key_id']=LNCEL['ecgiSourceEnbId']+'-'+LNCEL['lcrId']

In [156]:
LNCEL

,dn,version,name,eutraCelId,lcrId,administrativeState,mo,dn_parent,param_inco,action,check,netact,ecgiSourceEnbId,key_id
0,PLMN-PLMN/MRBTS-1002/LNBTS-1002/LNCEL-51,xL23R3_2309_110,22360_001-51,256563,51,1,LNCEL,PLMN-PLMN/MRBTS-1002/LNBTS-1002,,update,OK,NA1,1002,1002-51
1,PLMN-PLMN/MRBTS-1002/LNBTS-1002/LNCEL-52,xL23R3_2309_110,22360_001-52,256564,52,1,LNCEL,PLMN-PLMN/MRBTS-1002/LNBTS-1002,,update,OK,NA1,1002,1002-52
2,PLMN-PLMN/MRBTS-1002/LNBTS-1002/LNCEL-53,xL23R3_2309_110,22360_001-53,256565,53,1,LNCEL,PLMN-PLMN/MRBTS-1002/LNBTS-1002,,update,OK,NA1,1002,1002-53
3,PLMN-PLMN/MRBTS-1002/LNBTS-1002/LNCEL-61,xL23R3_2309_110,22360_001-61,256573,61,1,LNCEL,PLMN-PLMN/MRBTS-1002/LNBTS-1002,,update,OK,NA1,1002,1002-61
4,PLMN-PLMN/MRBTS-1002/LNBTS-1002/LNCEL-62,xL23R3_2309_110,22360_001-62,256574,62,1,LNCEL,PLMN-PLMN/MRBTS-1002/LNBTS-1002,,update,OK,NA1,1002,1002-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54226,PLMN-PLMN/MRBTS-4044655/LNBTS-404655/LNCEL-63,xL23R1_2221_110,nan,nan,nan,nan,LNCEL,PLMN-PLMN/MRBTS-4044655/LNBTS-404655,,update,OK,NA6,404655,404655-nan
54227,PLMN-PLMN/MRBTS-4044655/LNBTS-404655/LNCEL-72,xL23R1_2221_110,nan,nan,nan,nan,LNCEL,PLMN-PLMN/MRBTS-4044655/LNBTS-404655,,update,OK,NA6,404655,404655-nan
54228,PLMN-PLMN/MRBTS-4044655/LNBTS-404655/LNCEL-81,xL23R1_2221_110,nan,nan,nan,nan,LNCEL,PLMN-PLMN/MRBTS-4044655/LNBTS-404655,,update,OK,NA6,404655,404655-nan
54229,PLMN-PLMN/MRBTS-4044655/LNBTS-404655/LNCEL-82,xL23R1_2221_110,nan,nan,nan,nan,LNCEL,PLMN-PLMN/MRBTS-4044655/LNBTS-404655,,update,OK,NA6,404655,404655-nan


In [157]:
liste_EnbId=[]
for source_cell, value in liste_cells.items():
    source_id=LNCEL[LNCEL.name==source_cell].ecgiSourceEnbId.values[0]
    liste_EnbId.append(source_id)
    for target_cell in value : 
        try:
            print(value)
            print(target_cell)
            target_id=LNCEL[LNCEL.name==target_cell].ecgiSourceEnbId.values[0]
        except IndexError:
            pass;
        
        liste_EnbId.append(target_id)
liste_EnbId=list(set(liste_EnbId))

df_lnrel=pd.DataFrame()
for EnbId in liste_EnbId:
    df_1=get_lnrel(EnbId)   
    df_1['name_source']=df_1.dn_parent.map(pd.Series(LNCEL.name.values, index=LNCEL.dn).to_dict())
    
    df_1['name_target']=df_1['ecgiAdjEnbId']+'-'+df_1['ecgiLcrId']
    df_1.reset_index(drop=True,inplace=True)
    
    df_1['name_target']=df_1.name_target.map(pd.Series(LNCEL.name.values, index=LNCEL.key_id).to_dict())
    #df_1=pd.merge(df_1,LNCEL[["key_id","name"]],left_on="name_target",right_on="name",how='left')
    df_1['mo']='LNREL'
    print(df_1.head())
    df_lnrel=pd.concat([df_lnrel,df_1])
df_lnrel.drop_duplicates(inplace=True)
df_lnrel['cellIndOffNeigh']=df_lnrel['cellIndOffNeigh'].astype(int)

['69069_002-73', '69205_003-73', '69255_003-71']
69069_002-73
['69069_002-73', '69205_003-73', '69255_003-71']
69205_003-73
['69069_002-73', '69205_003-73', '69255_003-71']
69255_003-71
['69244_006-82', '69069_002-81', '69244_002-83']
69244_006-82
['69244_006-82', '69069_002-81', '69244_002-83']
69069_002-81
['69244_006-82', '69069_002-81', '69244_002-83']
69244_002-83
['69389_015-72', '69384_001-53', '69387_004-52']
69389_015-72
['69389_015-72', '69384_001-53', '69387_004-52']
69384_001-53
['69389_015-72', '69384_001-53', '69387_004-52']
69387_004-52
['69389_028-83', '69389_012-81', '69384_004-83']
69389_028-83
['69389_028-83', '69389_012-81', '69384_004-83']
69389_012-81
['69389_028-83', '69389_012-81', '69384_004-83']
69384_004-83
                                                  dn              version  \
0  PLMN-PLMN/MRBTS-403615/LNBTS-403615/LNCEL-63/L...      xL23R1_2221_110   
1  PLMN-PLMN/MRBTS-408116/LNBTS-408116/LNCEL-72/L...      xL23R1_2221_110   
2  PLMN-PLMN/MRBTS-420629

                                                  dn              version  \
0  PLMN-PLMN/MRBTS-1035/LNBTS-1035/LNCEL-53/LNREL...  xL23R3FSM3_2309_100   
1  PLMN-PLMN/MRBTS-1035/LNBTS-1035/LNCEL-53/LNREL...  xL23R3FSM3_2309_100   
2  PLMN-PLMN/MRBTS-1035/LNBTS-1035/LNCEL-53/LNREL...  xL23R3FSM3_2309_100   
3  PLMN-PLMN/MRBTS-1237/LNBTS-1237/LNCEL-53/LNREL...      xL23R3_2309_110   
4  PLMN-PLMN/MRBTS-1237/LNBTS-1237/LNCEL-81/LNREL...      xL23R3_2309_110   

  ecgiLcrId ecgiAdjEnbId cellIndOffNeigh handoverAllowed removeAllowed  \
0        62       406153              15               0             1   
1        51       406153              15               0             1   
2        61       406153              15               0             1   
3        53       406153              15               0             1   
4        81       406153              15               0             1   

                                  dn_parent param_inco  action check  \
0  PLMN-PLMN/MRBTS-1

                                                  dn              version  \
0  PLMN-PLMN/MRBTS-415801/LNBTS-415801/LNCEL-61/L...      xL23R3_2309_110   
1  PLMN-PLMN/MRBTS-410366/LNBTS-410366/LNCEL-81/L...      xL23R1_2221_110   
2  PLMN-PLMN/MRBTS-310983/LNBTS-310983/LNCEL-53/L...  xL23R1FSM3_2222_100   
3  PLMN-PLMN/MRBTS-16903/LNBTS-16903/LNCEL-52/LNR...      xL23R3_2309_110   
4  PLMN-PLMN/MRBTS-310366/LNBTS-310366/LNCEL-51/L...  xL23R1FSM3_2222_100   

  ecgiLcrId ecgiAdjEnbId cellIndOffNeigh handoverAllowed removeAllowed  \
0        61       404698              15               0             1   
1        61       404698              15               0             1   
2        61       404698              15               0             1   
3        61       404698              15               0             1   
4        61       404698              15               0             1   

                                      dn_parent param_inco  action check  \
0  PLMN-PLMN/MRB

In [158]:
LNREL_optim=pd.DataFrame(columns=df_lnrel.columns)
for key, value in liste_cells.items():
    for element in value :   
        print(key, element)
        df=df_lnrel[(df_lnrel.name_source==key) & (df_lnrel.name_target==element)]
        df['cellIndOffNeigh']=18
        df['param_inco']='cellIndOffNeigh'
        LNREL_optim=pd.concat([LNREL_optim,df], sort=False)
        df=df_lnrel[(df_lnrel.name_source==element) & (df_lnrel.name_target==key)]
        df['cellIndOffNeigh']=12
        df['param_inco']='cellIndOffNeigh'
        LNREL_optim=pd.concat([LNREL_optim,df], sort=False)
LNREL_optim['param_inco']=LNREL_optim['param_inco']+',handoverAllowed,removeAllowed'
LNREL_optim['handoverAllowed']='0'
LNREL_optim['removeAllowed']='0'

69069_003-71 69069_002-73
69069_003-71 69205_003-73
69069_003-71 69255_003-71
69202_004-83 69244_006-82
69202_004-83 69069_002-81
69202_004-83 69244_002-83
69389_015-52 69389_015-72
69389_015-52 69384_001-53
69389_015-52 69387_004-52
69389_015-81 69389_028-83
69389_015-81 69389_012-81
69389_015-81 69384_004-83


In [159]:
LNREL_optim.head()

,dn,version,ecgiLcrId,ecgiAdjEnbId,cellIndOffNeigh,handoverAllowed,removeAllowed,dn_parent,param_inco,action,check,ecgiSourceEnbId,name_source,name_target,mo
800,PLMN-PLMN/MRBTS-403414/LNBTS-403414/LNCEL-71/L...,xL23R1_2221_110,73,406153,18,0,0,PLMN-PLMN/MRBTS-403414/LNBTS-403414/LNCEL-71,"cellIndOffNeigh,handoverAllowed,removeAllowed",update,NOK,403414,69069_003-71,69069_002-73,LNREL
418,PLMN-PLMN/MRBTS-406153/LNBTS-406153/LNCEL-73/L...,xL23R1_2221_110,71,403414,12,0,0,PLMN-PLMN/MRBTS-406153/LNBTS-406153/LNCEL-73,"cellIndOffNeigh,handoverAllowed,removeAllowed",update,NOK,406153,69069_002-73,69069_003-71,LNREL
1035,PLMN-PLMN/MRBTS-403414/LNBTS-403414/LNCEL-71/L...,xL23R1_2221_110,73,420638,18,0,0,PLMN-PLMN/MRBTS-403414/LNBTS-403414/LNCEL-71,"cellIndOffNeigh,handoverAllowed,removeAllowed",update,NOK,403414,69069_003-71,69205_003-73,LNREL
457,PLMN-PLMN/MRBTS-420638/LNBTS-420638/LNCEL-73/L...,xL23R1_2221_110,71,403414,12,0,0,PLMN-PLMN/MRBTS-420638/LNBTS-420638/LNCEL-73,"cellIndOffNeigh,handoverAllowed,removeAllowed",update,NOK,420638,69205_003-73,69069_003-71,LNREL
650,PLMN-PLMN/MRBTS-403414/LNBTS-403414/LNCEL-71/L...,xL23R1_2221_110,71,422983,18,0,0,PLMN-PLMN/MRBTS-403414/LNBTS-403414/LNCEL-71,"cellIndOffNeigh,handoverAllowed,removeAllowed",update,NOK,403414,69069_003-71,69255_003-71,LNREL


In [160]:
if len(LNREL_optim):
    to_xml_liste_df([LNREL_optim],'cellIndOffNeigh_LNREL',name_xml)

In [161]:
LNREL_optim.to_excel('D:/LNREL_optim-dpt-sud-14-12.xlsx')